[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/da-python/python-for-data-analytics/blob/main/Lesson2/hometask/HH-api-hometask.ipynb)

# Задание
* Скачать все вакансии СБЕРа с ХХ (у апи есть ограничение в 2000, а также captcha подумайте как их можно обойти)
* Извлеките все описания этих вакансий
* Создайте аналогичный vacancy DataFrame только добавьте поле skills
* Переведите даты публикаций в datetime
* Постройте график опубликованных вакансий по датам
* Переведите даты в дни недели, и определите день недели, в который больше всего публикуют вакансий
* Найдите те вакансии с использованием python, которые вам интересны
* Определите по полю skills какие навыки больше всего востребованы для этих вакансий
* Постройте график наиболее востребованных навыков для аналитика данных

https://github.com/hhru/api

In [116]:
import requests
import json
from tqdm.auto import tqdm
from collections import defaultdict
import pickle
import pandas as pd
from dateutil.parser import parse

In [3]:
sber = '3529'
page = 1
num_per_page = 100
moscow = 1
url = f'https://api.hh.ru/vacancies?employer_id={sber}&page={page}&per_page={num_per_page}&area={moscow}'

In [4]:
res = requests.get(url)

In [6]:
res.json()

{'alternate_url': 'https://hh.ru/search/vacancy?area=1&control_flag=vacancyBlacklistJoinEnabled%3ATrue&employer_id=3529&enable_snippets=true&items_on_page=100&page=1',
 'arguments': None,
 'clusters': None,
 'found': 1184,
 'items': [{'accept_temporary': False,
   'address': {'building': '32к1',
    'city': 'Москва',
    'description': None,
    'id': '731097',
    'lat': 55.741338,
    'lng': 37.531127,
    'metro': {'lat': 55.740544,
     'line_id': '4',
     'line_name': 'Филевская',
     'lng': 37.5341,
     'station_id': '4.73',
     'station_name': 'Кутузовская'},
    'metro_stations': [{'lat': 55.740544,
      'line_id': '4',
      'line_name': 'Филевская',
      'lng': 37.5341,
      'station_id': '4.73',
      'station_name': 'Кутузовская'},
     {'lat': 55.740833,
      'line_id': '95',
      'line_name': 'МЦК',
      'lng': 37.533333,
      'station_id': '95.536',
      'station_name': 'Кутузовская'}],
    'raw': 'Москва, Кутузовский проспект, 32к1',
    'street': 'Кутузовск

In [7]:

vacancies = res.json()
num_pages = vacancies.get('pages')
num_pages

12

In [8]:
v = vacancies.get('items')

In [9]:
v[0]

{'accept_temporary': False,
 'address': {'building': '32к1',
  'city': 'Москва',
  'description': None,
  'id': '731097',
  'lat': 55.741338,
  'lng': 37.531127,
  'metro': {'lat': 55.740544,
   'line_id': '4',
   'line_name': 'Филевская',
   'lng': 37.5341,
   'station_id': '4.73',
   'station_name': 'Кутузовская'},
  'metro_stations': [{'lat': 55.740544,
    'line_id': '4',
    'line_name': 'Филевская',
    'lng': 37.5341,
    'station_id': '4.73',
    'station_name': 'Кутузовская'},
   {'lat': 55.740833,
    'line_id': '95',
    'line_name': 'МЦК',
    'lng': 37.533333,
    'station_id': '95.536',
    'station_name': 'Кутузовская'}],
  'raw': 'Москва, Кутузовский проспект, 32к1',
  'street': 'Кутузовский проспект'},
 'adv_response_url': 'https://api.hh.ru/vacancies/67978547/adv_response?host=hh.ru',
 'alternate_url': 'https://hh.ru/vacancy/67978547',
 'apply_alternate_url': 'https://hh.ru/applicant/vacancy_response?vacancyId=67978547',
 'archived': False,
 'area': {'id': '1', 'name'

In [10]:
df1 = pd.json_normalize(v)

In [11]:
df1.keys()

Index(['id', 'premium', 'name', 'has_test', 'response_letter_required',
       'salary', 'response_url', 'sort_point_distance', 'published_at',
       'created_at', 'archived', 'apply_alternate_url', 'insider_interview',
       'url', 'adv_response_url', 'alternate_url', 'relations', 'contacts',
       'working_days', 'working_time_intervals', 'working_time_modes',
       'accept_temporary', 'department.id', 'department.name', 'area.id',
       'area.name', 'area.url', 'type.id', 'type.name', 'address.city',
       'address.street', 'address.building', 'address.description',
       'address.lat', 'address.lng', 'address.raw',
       'address.metro.station_name', 'address.metro.line_name',
       'address.metro.station_id', 'address.metro.line_id',
       'address.metro.lat', 'address.metro.lng', 'address.metro_stations',
       'address.id', 'employer.id', 'employer.name', 'employer.url',
       'employer.alternate_url', 'employer.logo_urls.original',
       'employer.logo_urls.90', 'e

In [12]:
vacancies.keys()

dict_keys(['items', 'found', 'pages', 'per_page', 'page', 'clusters', 'arguments', 'alternate_url'])

In [13]:
vacancies = res.json()
num_pages = vacancies.get('pages')
vacancy_ids = [el.get('id') for el in vacancies.get('items')]

In [14]:
vacancies.keys()

dict_keys(['items', 'found', 'pages', 'per_page', 'page', 'clusters', 'arguments', 'alternate_url'])

In [15]:
vnum  = vacancies.get('pages')

In [16]:
all_vacancy_ids= []
for i in tqdm(range(vnum)):
    url = f'https://api.hh.ru/vacancies?employer_id={sber}&page={i}&per_page={num_per_page}&area={moscow}'
    res = requests.get(url)
    vacancies = res.json()
    vacancy_ids = [el.get('id') for el in vacancies.get('items')]
    all_vacancy_ids.extend(vacancy_ids)

  0%|          | 0/12 [00:00<?, ?it/s]

In [17]:
all_vacancy_ids

['53361761',
 '55415805',
 '55415496',
 '67349462',
 '67867647',
 '67861612',
 '45790891',
 '67887990',
 '67838397',
 '67608611',
 '67735776',
 '55889794',
 '67595487',
 '67659317',
 '67737477',
 '45381514',
 '55101523',
 '67525338',
 '67233281',
 '67595888',
 '67764112',
 '67472833',
 '67603928',
 '68003740',
 '67672404',
 '67716382',
 '67540818',
 '67917361',
 '66853831',
 '67889490',
 '68011820',
 '67908794',
 '66589132',
 '67353952',
 '67338661',
 '67535645',
 '67536591',
 '67409350',
 '66697224',
 '67807387',
 '67352204',
 '67504369',
 '67911334',
 '67740281',
 '67531354',
 '67513624',
 '66523856',
 '67511721',
 '67352254',
 '66534291',
 '67608480',
 '67313475',
 '67691816',
 '67649222',
 '66589225',
 '67852979',
 '68067583',
 '67706800',
 '67715431',
 '67385129',
 '67734236',
 '67636996',
 '66554589',
 '67548995',
 '67690176',
 '67996368',
 '67708223',
 '67553105',
 '67767117',
 '67264564',
 '67300477',
 '67670869',
 '67442824',
 '67362093',
 '67375671',
 '66912098',
 '67508678',

In [18]:
len(all_vacancy_ids)
#https://api.hh.ru/vacancies/67578966

1184

In [19]:
vacs=[]
for vac_id in tqdm(all_vacancy_ids):
  url = f'https://api.hh.ru/vacancies/{vac_id}'
  res = requests.get(url)
  vacs.append(res.json())


  0%|          | 0/1184 [00:00<?, ?it/s]

In [20]:
df2 = pd.json_normalize(vacs)

In [21]:
df2.iloc[0:3]

,id,premium,relations,name,insider_interview,response_letter_required,address,allow_messages,contacts,description,...,address.metro.station_name,address.metro.line_name,address.metro.station_id,address.metro.line_id,address.metro.lat,address.metro.lng,address.metro_stations,address.metro,test.required,immediate_redirect_url
0,53361761,False,[],Клиентский менеджер малого и микробизнеса,None,False,NaN,True,None,<p>Клиентский менеджер малого и микробизнеса –...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,55415805,False,[],Менеджер по работе с клиентами,None,False,NaN,True,None,<p>Сбер — лучший работодатель России. У нас бо...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,55415496,False,[],Клиентский менеджер прямых продаж,None,False,NaN,True,None,<p>Сбер — лучший работодатель России. У нас бо...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
url

'https://api.hh.ru/vacancies/67799535'

In [25]:
vrem

[{'name': 'Проведение презентаций'},
 {'name': 'Поиск и привлечение клиентов'},
 {'name': 'Холодные продажи'},
 {'name': 'Прямые продажи'}]

In [23]:
res

<Response [200]>

In [24]:
vrem = res.json().get('key_skills')
[x.get('name') for x in vrem]

['Проведение презентаций',
 'Поиск и привлечение клиентов',
 'Холодные продажи',
 'Прямые продажи']

In [61]:
dict_skills = {}

In [64]:
for i in range(len(df2['key_skills'])):
  dict_skills[df2['id'][i]] = [x.get('name') for x in df2['key_skills'][i]]

In [78]:
dict_skills

{'53361761': ['Активные продажи',
  'Проведение презентаций',
  'Привлечение клиентов',
  'Навыки продаж'],
 '55415805': [],
 '55415496': [],
 '67349462': [],
 '67867647': ['UX',
  'Figma',
  'Adobe Photoshop',
  'UI',
  'Дизайн интерфейсов',
  'Дизайн'],
 '67861612': ['JavaScript', 'Git', 'Redux', 'CSS', 'HTML'],
 '45790891': ['Прямые продажи',
  'Финансовый анализ',
  'Клиентоориентированность',
  'Развитие продаж',
  'Ведение переговоров',
  'Активные продажи',
  'Навыки продаж',
  'Проведение презентаций',
  'Работа с ключевыми клиентами',
  'Работа с юридическими лицами'],
 '67887990': ['Python', 'Pandas', 'PyTorch', 'Machine Learning'],
 '67838397': ['Работа в команде',
  'Грамотная речь',
  'Организаторские навыки',
  'Консультирование',
  'Обучение и развитие',
  'Внутренние коммуникации',
  'Деловая коммуникация',
  'Умение легко находить контакт с людьми',
  'Составление отчетности',
  'Способность быстро обучаться',
  'Уверенное знание ПК',
  'HR'],
 '67608611': [],
 '677357

In [74]:
df_skills = pd.DataFrame(list(dict_skills.items()),
                   columns=['id', 'skills'])
df_skills.head()

,id,skills
0,53361761,"[Активные продажи, Проведение презентаций, При..."
1,55415805,[]
2,55415496,[]
3,67349462,[]
4,67867647,"[UX, Figma, Adobe Photoshop, UI, Дизайн интерф..."


In [94]:
#df2.merge(df_skills)
dict_skills.get('67978903')

['Управление проектами',
 'Обучение и развитие',
 'Управление процессами',
 'Стратегическое мышление',
 'Лидерство',
 'Java',
 'JavaScript',
 'react',
 'Spring Framework']

In [84]:
df2['skills']=dict_skills.values()

In [95]:
df2.columns

Index(['id', 'premium', 'relations', 'name', 'insider_interview',
       'response_letter_required', 'address', 'allow_messages', 'contacts',
       'description', 'branded_description', 'vacancy_constructor_template',
       'key_skills', 'accept_handicapped', 'accept_kids', 'archived',
       'response_url', 'specializations', 'professional_roles', 'code',
       'hidden', 'quick_responses_allowed', 'driver_license_types',
       'accept_incomplete_resumes', 'published_at', 'created_at',
       'initial_created_at', 'negotiations_url', 'suitable_resumes_url',
       'apply_alternate_url', 'has_test', 'test', 'alternate_url',
       'working_days', 'working_time_intervals', 'working_time_modes',
       'accept_temporary', 'languages', 'billing_type.id', 'billing_type.name',
       'area.id', 'area.name', 'area.url', 'salary.from', 'salary.to',
       'salary.currency', 'salary.gross', 'type.id', 'type.name',
       'experience.id', 'experience.name', 'schedule.id', 'schedule.name',
  

In [117]:
parse(df2['published_at'][0])

datetime.datetime(2022, 6, 27, 12, 47, 8, tzinfo=tzoffset(None, 10800))

In [118]:
df2['published_at'][0]

'2022-06-27T12:47:08+0300'